# Example 5. Uncalibrated modulators: Oblique plane structured illumination microscopy

Load raw [oblique plane structured illumination microscopy (OPSIM)](https://www.biorxiv.org/content/10.1101/2022.05.19.492671v1.full), process data for rotation and registration, determine SIM pattern infromation, and run reconstruction.

Instrument designed and built by Bingying Chen, Bo-Jui Chang, and Reto Fiolka.  
Cardiomyocyte cell samples labeled for alpha-actinin 2 by James Hayes and Dylan Burnette.   
All imaging by Reto Fiolka. 

### Download data from Zenodo and extract

Data available from [OPSIM Zenodo repository](https://zenodo.org/record/6481084#.YmVM-7lOmHs).  Please download and extract "1_CH00_000000.tif" into data/example_005/raw_data. Once this is done, you can run this example.

### Import libraries

In [ ]:
import numpy as np
from numpy import fft
from pathlib import Path
import mcsim.analysis.sim_reconstruction as sim
from mcsim.analysis import analysis_tools
from localize_psf import rois
import matplotlib.pyplot as plt
from matplotlib.colors import PowerNorm
from skimage.transform import rotate, EuclideanTransform, warp
from skimage.registration import phase_cross_correlation
import napari
import tifffile
import gc
from example005_resources.deskew_opm_data import deskew
import itk

### Define data paths to raw OPSIM stack and 2D PSF extracted from many beads

In [ ]:
opsim_file_path = Path("data", "example_005", "raw_data", "1_CH00_000000.tif")
psf_file_path = Path("example005_resources", "psf2d.tif")

### Define experimental metadata

In [ ]:
# extracted from text file provided with acquisition
n_colors = 1
n_angles = 3
n_phases = 3
ns = 251
ny = 200
nx = 1024
pixel_size = 0.114 # um
ds = 0.320 # um - scan step size, not coverslip z
na = 1.0 # Snouty NA
excitation_wavelengths = 0.488 # um
emission_wavelengths = 0.580 # um
tilt_angle = 45.0

# transformed pixel spacing. qi2lab deskew deskews onto an isotropic grid in xy
coverslip_dz = pixel_size * np.sin(tilt_angle)
coverslip_dy = pixel_size
coverslip_dx = pixel_size

### Load and parse data into angles and phases

In [ ]:
# create array to hold all raw opsim images
sim_images = np.zeros((n_colors,n_angles,n_phases,ns,ny,nx),dtype=np.uint16)

# loop through TIFF file and load each raw opsim image into proper metadata location
page_idx = 0
with tifffile.TiffFile(opsim_file_path) as tif:
    n_pages = len(tif.pages)
    for channel_idx in range(n_colors):
        for angle_idx in range(n_angles):
            for phase_idx in range(n_phases):
                for scan_idx in range(ns):
                    sim_images[channel_idx,angle_idx,phase_idx,scan_idx,:,:] = tif.pages[page_idx].asarray()
                    page_idx = page_idx + 1

### Visualize OPM SIM data in native reference frame

In [ ]:
# add images to napari viewer with scale information
viewer = napari.view_image(sim_images,name='Raw OPSIM data',scale=(1,1,1,ds,pixel_size,pixel_size))

# activate scale bar in physical units
viewer.scale_bar.unit = 'um'
viewer.scale_bar.visible = True

# label sliders in napari viewer
viewer.dims.axis_labels = ['c','a','p','s','y','x']

### Calculate deskewed widefield equivalent images for registration
  
Orthogonal deskew function for OPM extracted from [qi2lab OPM package](https://www.github.com/qi2lab/opm). 

In [ ]:
# deskew one channel/angle/phase to get the size of the image
test_deskew = deskew(sim_images[0,0,0,:],tilt_angle,ds,pixel_size)
n_coverslip_z = test_deskew.shape[0]
n_coverslip_y = test_deskew.shape[1]
n_coverslip_x = test_deskew.shape[2]
coverslip_wf_images = np.zeros((n_colors,n_angles,n_coverslip_z,n_coverslip_y,n_coverslip_x),dtype=np.uint16)

# perform deskewing for all widefield equivalent views (average over phases)
for channel_idx in range(n_colors):
    for angle_idx in range(n_angles):
            image_plane = deskew(np.flipud(np.nanmean(sim_images[channel_idx,angle_idx,:,:,:,:],axis=0)),tilt_angle,ds,pixel_size)
            coverslip_wf_images[channel_idx,angle_idx,:] = image_plane

# clean up variables
del test_deskew
gc.collect()

### Rotate deskewed widefield equivalent images with prior info from instrument

In [ ]:
# pad images to be square
max_size = np.maximum(n_coverslip_y,n_coverslip_x)

if max_size > n_coverslip_y:
    pad_amount_y = np.abs(max_size-n_coverslip_y)//2
else:
    pad_amount_y = 0
if max_size > n_coverslip_x:
    pad_amount_x = np.abs(max_size-n_coverslip_x)//2
else:
    pad_amount_x = 0

pad_width = ((0,0),(0,0),(0,0),(pad_amount_y,pad_amount_y+1),(pad_amount_x,pad_amount_x))

coverslip_rotated_wf_images = np.pad(coverslip_wf_images,pad_width=pad_width)

# rotation angle guesses from instrument design
rotation = [60,0,-60]

# perform rotation across all z planes for each channel + angle
for channel_idx in range(n_colors):
    for angle_idx in range(n_angles):
        for z_idx in range(n_coverslip_z):
            rotated_plane = rotate(coverslip_rotated_wf_images[channel_idx,angle_idx,z_idx,:,:],
                                    angle=rotation[angle_idx],
                                    center=None,
                                    resize=False,
                                    mode='constant',
                                    cval=0,
                                    preserve_range=True)
            coverslip_rotated_wf_images[channel_idx,angle_idx,z_idx,:,:] = rotated_plane

# clean up variables
del coverslip_wf_images
gc.collect()

### Visualize rotated widefield equivalent images

In [ ]:
# add images to napari viewer with scale information
viewer = napari.view_image(coverslip_rotated_wf_images,name='Rotated Widefield equivalent',scale=(1,1,coverslip_dz,coverslip_dy,coverslip_dx))

# activate scale bar in physical units
viewer.scale_bar.unit = 'um'
viewer.scale_bar.visible = True

# label sliders in napari viewer
viewer.dims.axis_labels = ['c','a','z','y','x']

### Determine image registration in coverslip reference frame using deskewed and rotated widefield equivalent images

In [ ]:
# if ITK-elastix results are not satisfactory, set this flag to True to attempt an initial pre-registration using phase correlation
# initital value should be False
perform_initial_phase_correlation_registration = False

# optional phase correlation
if perform_initial_phase_correlation_registration:
    
    # Register using scikit-image
    # Create intial guess for translation based on phase-correlation and max projection images
    # +60 angle (angle_idx = 0) to 0 angle (angle_idx = 1)
    shifts_a000, _, _ = phase_cross_correlation(coverslip_rotated_wf_images[channel_idx,1,:],coverslip_rotated_wf_images[channel_idx,0,:])
    tform_a000 = EuclideanTransform(
        rotation= 0,
        translation = (shifts_a000[2], shifts_a000[1])
        )
    for z_idx in range(n_coverslip_z):
        coverslip_rotated_wf_images[channel_idx,0,z_idx,:]=warp(coverslip_rotated_wf_images[channel_idx,0,z_idx,:], tform_a000.inverse, preserve_range=True)

    # -60 angle (angle_idx = 2) to 0 angle (angle_idx = 1)
    shifts_a002, _, _ = phase_cross_correlation(coverslip_rotated_wf_images[channel_idx,1,:],coverslip_rotated_wf_images[channel_idx,2,:])
    tform_a002 = EuclideanTransform(
        rotation= 0,
        translation = (shifts_a002[2], shifts_a002[1])
        )
    for z_idx in range(n_coverslip_z):
        coverslip_rotated_wf_images[channel_idx,2,z_idx,:]=warp(coverslip_rotated_wf_images[channel_idx,2,z_idx,:], tform_a002.inverse, preserve_range=True)

# Register using ITK-elastix
# create ITK array for fixed view
# itk-elastix currently supports np.float32 only
channel_idx = 0
angle_idx = 1
fixed_image_a001 = itk.GetImageFromArray(coverslip_rotated_wf_images[channel_idx,angle_idx,:].astype(np.float32))

# first, perform translation registration
parameter_object = itk.ParameterObject.New()
default_translation_parameter_map = parameter_object.GetDefaultParameterMap('translation')
default_translation_parameter_map['FinalBSplineInterpolationOrder'] = ['2']
parameter_object.AddParameterMap(default_translation_parameter_map)

# second, perform rigid registration
default_rigid_parameter_map = parameter_object.GetDefaultParameterMap('rigid')
default_rigid_parameter_map['FinalBSplineInterpolationOrder'] = ['2']
parameter_object.AddParameterMap(default_rigid_parameter_map)

# third, perform affine registration
default_affine_parameter_map = parameter_object.GetDefaultParameterMap('affine')
default_affine_parameter_map['FinalBSplineInterpolationOrder'] = ['2']
parameter_object.AddParameterMap(default_affine_parameter_map)

# Call registration functions to determine registration of rotated view (angle_idx=[0,2]) to center view (angle_idx=1)

# create ITK array for +60 rotated view
# itk-elastix currently supports np.float32 only
angle_idx = 0
moving_image_a000 = itk.GetImageFromArray(coverslip_rotated_wf_images[channel_idx,angle_idx,:].astype(np.float32))

# run translation->rigid->affine registration for angle_idx=0
result_image_a000, result_transform_parameters_a000_to_a001 = itk.elastix_registration_method(
    fixed_image_a001, moving_image_a000,
    parameter_object=parameter_object,
    log_to_console=False)

# clean up variables for +60 rotated view
del moving_image_a000
gc.collect()

# create ITK array for -60 rotated view
# itk-elastix currently supports np.float32 only
angle_idx = 2
moving_image_a002 = itk.GetImageFromArray(coverslip_rotated_wf_images[channel_idx,angle_idx,:].astype(np.float32))

# run translation->rigid->affine registration for angle_idx=2
result_image_a002, result_transform_parameters_a002_to_a001 = itk.elastix_registration_method(
    fixed_image_a001, moving_image_a002,
    parameter_object=parameter_object,
    log_to_console=False)

# clean up variables for -60 rotated view
del moving_image_a002
gc.collect()


### Visualize results of registration on deskewed widefield equivalent images
If registration results are not satisfactory, change `perform_initial_phase_correlation_registration` in above code block to `True` and re-run registration.

In [ ]:
# add images to napari viewer with scale information
viewer = napari.view_image(result_image_a000,name='Registered deskewed angle 0',scale=(coverslip_dz,coverslip_dy,coverslip_dx),blending='additive',contrast_limits=[0,np.percentile(result_image_a000, 99.99)])
viewer.add_image(fixed_image_a001,name='Registered deskewed angle 1',scale=(coverslip_dz,coverslip_dy,coverslip_dx),blending='additive',contrast_limits=[0,np.percentile(fixed_image_a001, 99.99)])
viewer.add_image(result_image_a002,name='Registered deskewed angle 2',scale=(coverslip_dz,coverslip_dy,coverslip_dx),blending='additive',contrast_limits=[0,np.percentile(result_image_a002, 99.99)])

# activate scale bar in physical units
viewer.scale_bar.unit = 'um'
viewer.scale_bar.visible = True

# label sliders in napari viewer
viewer.dims.axis_labels = ['z','y','x']

### Deskew all raw OPSIM images

In [ ]:
# deskew one channel/angle/phase to get the size of the image
test_deskew = deskew(sim_images[0,0,0,:],tilt_angle,ds,pixel_size)
n_coverslip_z = test_deskew.shape[0]
n_coverslip_y = test_deskew.shape[1]
n_coverslip_x = test_deskew.shape[2]

# create array to hold all deskewed images
coverslip_opsim_images = np.zeros((n_colors,n_angles,n_phases,n_coverslip_z,n_coverslip_y,n_coverslip_x),dtype=np.uint16)

# perform deskewing for all raw opsim views
for channel_idx in range(n_colors):
    for angle_idx in range(n_angles):
        for phase_idx in range(n_phases):
            coverslip_opsim_images[channel_idx,angle_idx,phase_idx,:] = deskew(np.flipud(sim_images[channel_idx,angle_idx,phase_idx,:,:,:]),tilt_angle,ds,pixel_size)

# clean up variables
del test_deskew, sim_images
gc.collect()

### Rotate deskewed raw OPSIM images

In [ ]:
# pad images to be square to hold rotated images
max_size = np.maximum(n_coverslip_y,n_coverslip_x)

if max_size > n_coverslip_y:
    pad_amount_y = np.abs(max_size-n_coverslip_y)//2
else:
    pad_amount_y = 0
if max_size > n_coverslip_x:
    pad_amount_x = np.abs(max_size-n_coverslip_x)//2
else:
    pad_amount_x = 0

pad_width = ((0,0),(0,0),(0,0),(0,0),(pad_amount_y,pad_amount_y+1),(pad_amount_x,pad_amount_x))
rotated_coverslip_opsim_images = np.pad(coverslip_opsim_images,pad_width=pad_width)

# rotation angle guesses from instrument design
rotation = [60,0,-60]

# perform rotation across all z planes for each channel + angle
for channel_idx in range(n_colors):
    for angle_idx in range(n_angles):
        for phase_idx in range(n_phases):
            for z_idx in range(n_coverslip_z):
                rotated_plane = rotate(rotated_coverslip_opsim_images[channel_idx,angle_idx,phase_idx,z_idx,:,:],
                                        angle=rotation[angle_idx],
                                        center=None,
                                        resize=False,
                                        mode='constant',
                                        cval=0,
                                        preserve_range=True)
                rotated_coverslip_opsim_images[channel_idx,angle_idx,phase_idx,z_idx,:,:] = rotated_plane

# clean up variables
del coverslip_opsim_images
gc.collect()

### Register deskewed raw OPSIM images using transformation matrices found using deskewed widefield equivalent images

In [ ]:
# create temporary variable to hold results of initial translation warp
result_image_translation = np.zeros_like(rotated_coverslip_opsim_images[0,0,0,:])

# loop over all channel/angle/phase and apply transformations for angle_idx=[0,2] to angle_idx=1
for channel_idx in range(n_colors):
    for angle_idx in range(n_angles):
        for phase_idx in range(n_phases):
            if angle_idx == 0:
                if perform_initial_phase_correlation_registration:
                    for z_idx in range(n_coverslip_z):
                        result_image_translation[z_idx,:]=warp(rotated_coverslip_opsim_images[channel_idx,angle_idx,phase_idx,z_idx,:,:], tform_a000.inverse, preserve_range=True)
                    result_image_transformix = itk.transformix_filter(result_image_translation.astype(np.float32),result_transform_parameters_a000_to_a001)
                else:
                    result_image_transformix = itk.transformix_filter(rotated_coverslip_opsim_images[channel_idx,angle_idx,phase_idx,:].astype(np.float32),result_transform_parameters_a000_to_a001)
                rotated_coverslip_opsim_images[channel_idx,angle_idx,phase_idx,:,:,:]=result_image_transformix.astype(np.uint16)
            elif angle_idx == 2:
                if perform_initial_phase_correlation_registration:
                    for z_idx in range(n_coverslip_z):
                        result_image_translation[z_idx,:]=warp(rotated_coverslip_opsim_images[channel_idx,angle_idx,phase_idx,z_idx,:,:], tform_a002.inverse, preserve_range=True)
                    result_image_transformix = itk.transformix_filter(result_image_translation.astype(np.float32),result_transform_parameters_a002_to_a001)
                else:
                    result_image_transformix = itk.transformix_filter(rotated_coverslip_opsim_images[channel_idx,angle_idx,phase_idx,:].astype(np.float32),result_transform_parameters_a002_to_a001)
                rotated_coverslip_opsim_images[channel_idx,angle_idx,phase_idx,:,:,:]=result_image_transformix.astype(np.uint16)

del result_image_translation, result_image_transformix

### Visualize registered deskewed OPSIM images in coverslip reference frame

In [ ]:
# add images to napari viewer with scale information
viewer = napari.view_image(rotated_coverslip_opsim_images[0,0,:],name='Registered deskewed angle 0',scale=(1,coverslip_dz,coverslip_dy,coverslip_dx),blending='additive',contrast_limits=[0,np.percentile(rotated_coverslip_opsim_images[0,0,:], 99.99)])
viewer.add_image(rotated_coverslip_opsim_images[0,1,:],name='Registered deskewed angle 1',scale=(1,coverslip_dz,coverslip_dy,coverslip_dx),blending='additive',contrast_limits=[0,np.percentile(rotated_coverslip_opsim_images[0,1,:], 99.99)])
viewer.add_image(rotated_coverslip_opsim_images[0,2,:],name='Registered deskewed angle 2',scale=(1,coverslip_dz,coverslip_dy,coverslip_dx),blending='additive',contrast_limits=[0,np.percentile(rotated_coverslip_opsim_images[0,2,:], 99.99)])

# activate scale bar in physical units
viewer.scale_bar.unit = 'um'
viewer.scale_bar.visible = True

# label sliders in napari viewer
viewer.dims.axis_labels = ['c','a','p','z','y','x']

### Set ROI to middle of registered deskewed OPIM images and extract crop

In [ ]:
# set yx width of ROI
roi_sy=429
roi_sx=449
roi = rois.get_centered_roi([rotated_coverslip_opsim_images.shape[4]//2, rotated_coverslip_opsim_images.shape[5]//2], [roi_sy, roi_sx])

# set these using the z slices that contain data from the Napari preview window above
z_roi_low = 25
z_roi_high = 95

# extract crop
opsim_images_to_process = rotated_coverslip_opsim_images[:,:,:,z_roi_low:z_roi_high,roi[0]:roi[1],roi[2]:roi[3]]

# extract shape of cropped data
nz_cropped = opsim_images_to_process.shape[3]
ny_cropped = opsim_images_to_process.shape[4]
nx_cropped = opsim_images_to_process.shape[5]

### Visualize cropped registered deskewed OPSIM images to pick "best" slice

In [ ]:
# add images to napari viewer with scale information
viewer = napari.view_image(opsim_images_to_process[0,0,:],name='Registered deskewed angle 0',scale=(1,coverslip_dz,coverslip_dy,coverslip_dx),blending='additive',contrast_limits=[0,np.percentile(opsim_images_to_process[0,0,:], 99.99)])
viewer.add_image(opsim_images_to_process[0,1,:],name='Registered deskewed angle 1',scale=(1,coverslip_dz,coverslip_dy,coverslip_dx),blending='additive',contrast_limits=[0,np.percentile(opsim_images_to_process[0,1,:], 99.99)])
viewer.add_image(opsim_images_to_process[0,2,:],name='Registered deskewed angle 2',scale=(1,coverslip_dz,coverslip_dy,coverslip_dx),blending='additive',contrast_limits=[0,np.percentile(opsim_images_to_process[0,2,:], 99.99)])

# activate scale bar in physical units
viewer.scale_bar.unit = 'um'
viewer.scale_bar.visible = True

# label sliders in napari viewer
viewer.dims.axis_labels = ['c','a','p','z','y','x']

### Find SIM peaks for each angle
See [this section of the I2K mcSIM tutorial](https://youtu.be/mDar-MjMtW0?t=6421) video and our [Nikon N-SIM processing tutorial](https://github.com/QI2lab/I2K2022-SIM/blob/0660281c28a809fdc09c36b00903b9230feb4d9c/example_004_Nikon_NSIM.ipynb) for how use the code block to find SIM frequencies.

In [ ]:
%matplotlib widget

# set "test" slice as determined from data
# chose a z slice with lots of data so that the SIM peaks are visible
z_test_idx=22

for ch_idx in range(n_colors):

    # extract iamges
    image_set = opsim_images_to_process[ch_idx,:,:,z_test_idx,:,:]

    # calculate FFTs on grid with correct spatial dimensions
    dx = np.round(coverslip_dx,3)
    dy = np.round(coverslip_dy,3)
    nx = nx_cropped
    ny = ny_cropped
    fxs = analysis_tools.get_fft_frqs(nx, dx)
    df = fxs[1] - fxs[0]
    fys = analysis_tools.get_fft_frqs(ny, dy)
    ff = np.sqrt(np.expand_dims(fxs, axis=0)**2 + np.expand_dims(fys, axis=1)**2)
    for ii in range(image_set.shape[0]):
        ft = fft.fftshift(fft.fft2(fft.ifftshift(np.squeeze(image_set[ii, 0, :]))))

        figh = plt.figure()
        plt.title('Channel='+str(ch_idx)+", Angle="+str(ii))
        plt.imshow(np.abs(ft), norm=PowerNorm(gamma=0.1,vmin=65,vmax=1.5e7),
                    extent=[fxs[0] - 0.5 * df, fxs[-1] + 0.5 * df, fys[-1] + 0.5 * df, fys[0] - 0.5 * df])
        plt.show()

### Setup 2D-SIM processing

It is critical to set the initial guesses for SIM peaks correctly in `frqs_guess`. Double check results using diagnostic output plots.

In [ ]:
# frequency guess from above. in k_x, k_y order
# getting these as close as possible is critical to reconstruction quality!
frq_guess = [[-1.33,2.28], [-2.63,0.005], [-1.29,-2.28]]

# phase guesses from prior knowledge
phase_guess = [[0, -4*np.pi/3,-2*np.pi/3], [0, -4*np.pi/3,-2*np.pi/3], [0, -4*np.pi/3,-2*np.pi/3]]

# algorithm parameters
# These need to be tuned for each OPSIM dataset
wiener_parameter = 0.35
band0_exclusion_fraction = 0.5

### Load point spread function and calculate optical transfer function for each angle

2D PSF extracted from OPSIM data of beads by Reto Fiolka and Peter Brown using [qi2lab localize-psf package](https://www.github.com/qi2lab/localize-psf).  

In [ ]:
# load PSF from disk and tranpose to match metadata from raw data
psf2d = np.transpose(tifffile.imread(psf_file_path))
ny_psf, nx_psf = psf2d.shape

# create one PSF for each angle
psfs = np.asarray([rotate(psf2d, rotation[0], center=None, resize=False, mode='constant', cval=0, preserve_range=True),
                   psf2d,
                   rotate(psf2d, rotation[2], center=None, resize=False, mode='constant', cval=0, preserve_range=True)])

# pad to get size of image
if np.mod(ny_psf + ny_cropped, 2) != 0 or np.mod(nx_psf + nx_cropped, 2) != 0:
    raise ValueError()

# pad each PSF
ny_pad = int((ny_cropped - ny_psf) / 2)
nx_pad = int((nx_cropped - nx_psf) / 2)
psfs_padded = []
for p in psfs:
    psfs_padded.append(np.pad(p, ((ny_pad, ny_pad), (nx_pad, nx_pad)), mode="constant"))
psfs_padded = np.asarray(psfs_padded)

# get spatial frequencies on correct grid
fxs = fft.fftshift(fft.fftfreq(nx_cropped, coverslip_dx))
df = fxs[1] - fxs[0]
fys = fft.fftshift(fft.fftfreq(ny_cropped, coverslip_dy))
ff = np.sqrt(np.expand_dims(fxs, axis=0)**2 + np.expand_dims(fys, axis=1)**2)

# create OTF for each angle
fmax = 1 / (0.5 * emission_wavelengths / na)
otfs = []
for p in psfs_padded:
    otf = fft.fftshift(fft.fft2(fft.ifftshift(p)))
    otf = otf / np.max(np.abs(otf))
    otf[np.abs(otf) < 0.005] = 0
    otf[ff > fmax] = 0
    otf = np.abs(otf)

    otfs.append(otf)
otfs = np.asarray(otfs)

### Run 2D-SIM processing for test z slice and extract reconstruction parameters
  
Modifications to [qi2lab mcSIM package](https://www.github.com/qi2lab/mcSIM) to handle OPSIM data by Peter Brown.  
  
Ouputs diagnostic plots in "reconstruction/test_z_plane" directory

In [ ]:
# define save path
save_path = Path("data", "example_005", "reconstruction", "test_z_plane")

# create SIM reconstruction object for z_center_idx 
imgset_test = sim.SimImageSet({'pixel_size': coverslip_dx, 'na': na, 'wavelength': emission_wavelengths},
                               opsim_images_to_process[0, :, :, z_test_idx, :],
                               frq_guess=frq_guess,
                               phases_guess=phase_guess,
                               wiener_parameter=wiener_parameter,
                               phase_estimation_mode="wicker-iterative",
                               max_phase_err=40 * np.pi / 180,
                               combine_bands_mode="fairSIM", 
                               otf=otfs,
                               fmax_exclude_band0=band0_exclusion_fraction,
                               min_p2nr=0.9,
                               gain=1, 
                               background=100,
                               save_dir=save_path,
                               save_suffix='_z'+str(z_test_idx).zfill(3)+'_ch'+str(0).zfill(3),
                               interactive_plotting=False)

# perform reconstruction, plot figures, save
imgset_test.reconstruct()
imgset_test.plot_figs()
imgset_test.save_imgs()
imgset_test.save_result()

# save reconstruction parameter for use on full stack
frqs = np.array(imgset_test.frqs, copy=True)
phases = np.array(imgset_test.phases, copy=True)
mod_depths = np.array(imgset_test.mod_depths, copy=True)

wf_image_test = imgset_test.widefield
SR_image_test = imgset_test.sim_sr

# clean up variables
del imgset_test
gc.collect()

### Visualize test slice reconstruction

In [ ]:
# define colormaps
colormaps = ['bop blue', 'bop orange']

# add images to napari viewer with scale information, colormaps, and additive blending
viewer = napari.view_image(wf_image_test,name='Widefield test',scale=(coverslip_dy,coverslip_dx),colormap = colormaps[0],blending='additive',contrast_limits=[0,np.percentile(wf_image_test, 99.9)])
viewer.add_image(SR_image_test,name='SIM test',scale=(coverslip_dy/2,coverslip_dx/2),colormap = colormaps[1],blending='additive',contrast_limits=[0,np.percentile(SR_image_test,99.9)])

# activate scale bar in physical units
viewer.scale_bar.unit = 'um'
viewer.scale_bar.visible = True

### Reconstruct full z stack of cropped registered deskewed OPSIM images using parameters from selected z plane

In [ ]:
# define save path
save_path = Path("data", "example_005", "reconstruction", "full_z_stack")

# loop over channels and z slices, perform 2D at each slice using parameters from above reconstruction
for ch_idx in range(n_colors):
    for z_idx in range(nz_cropped):
        imgs_recon = opsim_images_to_process[ch_idx, :, :, z_idx,:,:]

        imgset = sim.SimImageSet({'pixel_size': coverslip_dx, 'na': na, 'wavelength': emission_wavelengths}, 
                                imgs_recon,
                                frq_guess=frqs,
                                frq_estimation_mode="fixed",
                                phases_guess=phases,
                                phase_estimation_mode="fixed",
                                max_phase_err=40 * np.pi / 180,
                                mod_depths_guess=mod_depths,
                                use_fixed_mod_depths=True,
                                wiener_parameter=wiener_parameter,
                                combine_bands_mode="fairSIM",
                                fmax_exclude_band0=band0_exclusion_fraction,
                                otf=otfs,
                                gain=1, 
                                background=100,
                                save_dir=save_path, 
                                save_suffix='_z'+str(z_idx).zfill(3)+'_ch'+str(0).zfill(3),
                                interactive_plotting=False)
        imgset.reconstruct()
        imgset.save_result()
        imgset.save_imgs()
        imgset.plot_figs()

        # create variables to hold widefield and SIM SR images
        if ch_idx == 0 and z_idx == 0:
            wf_images = np.zeros((n_colors,nz_cropped,imgset.widefield.shape[0],imgset.widefield.shape[1]),dtype=np.float32)
            SR_images = np.zeros((n_colors,nz_cropped,imgset.sim_sr.shape[0],imgset.sim_sr.shape[1]),dtype=np.float32)

        # store widefield and SIM SR images for display
        wf_images[ch_idx,z_idx,:]=imgset.widefield
        SR_images[ch_idx,z_idx,:]=imgset.sim_sr

        # clean up variables
        del imgs_recon, imgset
        gc.collect()

### Visualize z stack SIM reconstruction

In [ ]:
# define colormaps
colormaps = ['bop blue', 'bop orange']

# add images to napari viewer with scale information, colormaps, and additive blending
viewer = napari.view_image(wf_images[ch_idx,:],name='Widefield CH '+str(ch_idx).zfill(2),scale=(coverslip_dz,coverslip_dy,coverslip_dx),colormap = colormaps[0],blending='additive',contrast_limits=[0,np.percentile(wf_images[ch_idx,:], 99.9)])
viewer.add_image(SR_images[ch_idx,:],name='SR CH '+str(ch_idx).zfill(2),scale=(coverslip_dz,coverslip_dy/2,coverslip_dx/2),colormap = colormaps[1],blending='additive',contrast_limits=[0,np.percentile(SR_images[ch_idx,:], 99.99)])

# activate scale bar in physical units
viewer.scale_bar.unit = 'um'
viewer.scale_bar.visible = True

### Write widefield equivalent and SIM reconstructions to disk

In [ ]:
# loop over all channels
for ch_idx in range(n_colors):
  # widefield
  save_path_wf = Path("data", "example_005", "reconstruction", "ch_"+str(ch_idx).zfill(3)+"_reconstruction_wf.tif")
  tifffile.imwrite(save_path_wf,
                  wf_images[ch_idx,:], 
                  imagej=True, 
                  resolution=(1./(coverslip_dx), 1./(coverslip_dy)), 
                  metadata={'spacing': coverslip_dz, 'unit': 'um', 'min': 0,'max': 2**16-1,'axes': 'ZYX'})

  # opsim
  save_path_opsim = Path("data", "example_005", "reconstruction", "ch_"+str(ch_idx).zfill(3)+"_reconstruction_opsim.tif")
  tifffile.imwrite(save_path_opsim, 
                  SR_images[ch_idx,:], 
                  imagej=True, 
                  resolution=(1./(coverslip_dx/2), 1./(coverslip_dy/2)), 
                  metadata={'spacing': coverslip_dz, 'unit': 'um', 'min': 0,'max': 2**16-1,'axes': 'ZYX'})